# This NB was created to generate primers for KOing genes in yeast by replacing the gene with KanMX
#with user cloning instead of overlap fragments


In [ ]:
The first part of the NB is the same as USER_Primers: it design user primers to anneal geneUP::KanMX::geneDw together
#the second part uses elements from Yeast KO to design confirmation primers



In [10]:
import numpy as np
import pandas as pd
import  re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.SeqUtils.MeltingTemp import make_table, DNA_NN2
import pyperclip
import importlib
import DNA_tools as dt

import os
import  Bio

from Bio import SeqIO


#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
Breslauer= mt.DNA_NN1
Santalucia= mt.DNA_NN4 #same as benchling
Tm_Opt =Santalucia

In [2]:
#forward seq of user overhangs:
User_Overhangs={}
User_Overhangs['GV1']= 'CGTGCGAu'
User_Overhangs['GV2']= 'ATCGCGTG'#'CACGCGAu'
User_Overhangs['U1'] = 'AGCCCTCTu'
User_Overhangs['U2'] = 'ACACAGGCu'
User_Overhangs['U3'] = 'ATGCCACCu'
User_Overhangs['U4'] = 'AAGGAGCGu'
User_Overhangs['U5'] = 'ACCGTAGCu'
User_Overhangs['U6'] = 'ACTGCGAGu'
User_Overhangs['U7'] = 'AGTGCAGGu'
User_Overhangs['none'] = 'TATATATATATATA'

In [3]:
#for mads: insertion of BTX in YER084W

YER084W_up={}
YER084W_up['name']='YER084W_CDS'
YER084W_up['seq']='ggtctttgcagcgctaggagcgtctaaaggtgattctgcgttcaaatgactgcttgcctgcccggtgattttattaagcctggagctggcacctccattgctaaacttcttctgtctcctttcccttagtaacctgcgcttttccgcctctgttaattcagacatatttcctagagctacgttgctgatgctacaaaca'
YER084W_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_up['Primer_up_U']='GV1' #user site up
YER084W_up['Primer_down_U']='U1'#user site down
YER084W_up['Primer_up']=''
YER084W_up['Primer_up_Tm']=''
YER084W_up['Primer_down']=''
YER084W_up['Primer_down_Tm']=''

BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='ATTAAGTCCTCAGCGAGCTCGCATGGAATGCGTGCGATGAGCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTGATATCAGATCCACTAGTG'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='U1' #user site up
BTX['Primer_down_U']='U2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''

YER084W_dw={}
YER084W_dw['name']='YER084W_CDS'
YER084W_dw['seq']='taactcagattttcaagaaacagaatagcacattctctaacatcaagttatcataccaatatataacacaaattatgcataaatagataagtaatatgtccctctgcccctgcctttccctttccagatgctggcatcgaaagtaaggcatgtatttgtatgtaagttttttaattaatagtcttcttctttcttatcag'
YER084W_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_dw['Primer_up_U']='U2' #user site up
YER084W_dw['Primer_down_U']='GV2'#user site down
YER084W_dw['Primer_up']=''
YER084W_dw['Primer_up_Tm']=''
YER084W_dw['Primer_down']=''
YER084W_dw['Primer_down_Tm']=''

YER084W_KO_BTX=[YER084W_up,BTX,YER084W_dw]

In [201]:
### Design the primers for addind CDSs to the BTX pathway + making the KO:
###up regions (promoter): 1000bp upstream of CDS unless another sequeunce is there
### upstream is 500bp regardless of presence of other CDS (might include the end of another gene)
### for PDX3 promoter is close to snoRNA, so only took 240bp long promoter
### for PDX3 terminator is close to start of other CDS so took a shorter terminator
##down region (terminator): 500dp dwnstream of CDS unless another sequeunce is there

#for the KO primers (reverse of up and forward of down) to avoid nesting promoter>>promoter::KANMX::terminator>>terminator
#end up region so that total length is 200bp
#start dwn region so that total length is 200bp

#YDL119C =HEM25
#KGD2 v
#PPA1 = VMA16
#CAX4
#Pro2
#GLR1
#YSY6
#PDX3

#BTX pathway will be annealed straight into pCCM021 to be flanked by XII-up and  HomoB
BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='GCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAAT'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='GV1' #user site up
BTX['Primer_down_U']='GV2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''


## CDSs ups and downs regions will be linked to KanMx to generate KO plasmids
KanMX={}
KanMX['name']='pTEF::KanMX::tTef'
KanMX['seq']='TAGGTCTAGAGATCTGTTTAGCttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgaGCTCTCGAGAACCCTTAAT'
KanMX['need_reverse']=0 #if parts needs to be reversed:1 else:0
KanMX['Primer_up_U']='U1' #user site up
KanMX['Primer_down_U']='U2'#user site down
KanMX['Primer_up']=''
KanMX['Primer_up_Tm']=''
KanMX['Primer_down']=''
KanMX['Primer_down_Tm']=''



##in addition,CDS will be PCRed with the forward primer of the Up and the reverse of the down
#and annealed into pCCM022 to be flanked by  HomoB and XII-dwn

KGD2_up={}
KGD2_up['name']='KGD2_up'
KGD2_up['seq']='ggcaggaccttagttttgattggctgcaaaaaaagggttgttaatatcgtctttgacggagatgcgggtggctggaattgttttgtttattattccccaagagagaagagacaagaagtagtaatgacgatggaacaaagatcggttctaataggaccacagggttgaatagagacgagtaaagacgcaacggtcaaggctttccacgagacgttcgaat'
KGD2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_up['Primer_up_U']='GV1' #user site up
KGD2_up['Primer_down_U']='U1'#user site down
KGD2_up['Primer_up']=''
KGD2_up['Primer_up_Tm']=''
KGD2_up['Primer_down']=''
KGD2_up['Primer_down_Tm']=''

KGD2_dw={}
KGD2_dw['name']='KGD2_dw'
KGD2_dw['seq']='atgtggaaatcaaaagaatattagttgattctattatataataaatagttataatcattgtattttgatgatgaaaattcagtatctaagagttatagccaagagagtaatttttaatacttttttgcgcttttccatatctcattataaaaagaaaacattaattcatagttccatttatcatcatagcatatctttaa'
KGD2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_dw['Primer_up_U']='U2' #user site up
KGD2_dw['Primer_down_U']='GV2'#user site down
KGD2_dw['Primer_up']=''
KGD2_dw['Primer_up_Tm']=''
KGD2_dw['Primer_down']=''
KGD2_dw['Primer_down_Tm']=''



HEM25_up={}
HEM25_up['name']='HEM25_up'
HEM25_up['seq']='atttcacttcgtccttaatattttaacacattaactgtttgctataagcatacattgcttttaccactttttttaataatatctttttatccgtattaattaccgcatataattttagttcgaaacattaaaggtcagtcccgaagttataattataacatttaattcctgaatttaccgtattacatttaaatttgcat'
HEM25_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_up['Primer_up_U']='GV1' #user site up
HEM25_up['Primer_down_U']='U1'#user site down
HEM25_up['Primer_up']=''
HEM25_up['Primer_up_Tm']=''
HEM25_up['Primer_down']=''
HEM25_up['Primer_down_Tm']=''

HEM25_dw={}
HEM25_dw['name']='HEM25_dw'
HEM25_dw['seq']='gtgcacttatccatcgactctcttatatttatttaaacacttgtaacgcatccaatctatttcatagttagagtctctcttccttgttattgatatgatttttcttttgtaccgtgtgtgtatgtatgtatgtatgaacgtactactaagcttcaacgtcactctataaagaaactaaaaagacacgtatttatacctt'
HEM25_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_dw['Primer_up_U']='U2' #user site up
HEM25_dw['Primer_down_U']='GV2'#user site down
HEM25_dw['Primer_up']=''
HEM25_dw['Primer_up_Tm']=''
HEM25_dw['Primer_down']=''
HEM25_dw['Primer_down_Tm']=''






VMA16_up={} #PPA1
VMA16_up['name']='VMA16_up'
VMA16_up['seq']='ggcctattgggtatctgcttgtcaggtgcaggcccaactatcttggctttggccactgagaatttcgaagaaatctctcaagaaattatcaacaggttcgccaaaaacggcatcaagtgctcctggaaactactggagcctgcctacgatggtgctagcgtcgaacagcaatgaaatgtcctttacccctagtatactgtgtt'
VMA16_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_up['Primer_up_U']='GV1' #user site up
VMA16_up['Primer_down_U']='U1'#user site down
VMA16_up['Primer_up']=''
VMA16_up['Primer_up_Tm']=''
VMA16_up['Primer_down']=''
VMA16_up['Primer_down_Tm']=''

VMA16_dw={}
VMA16_dw['name']='VMA16_dw'
VMA16_dw['seq']='agcgcatccatatttactaatagatataatgagatatctgagatacatgtgtatgtatatatgcacgttttcttttattatctaaaaatcatattatattaagtaagagaaaaaaatgtacaactatataaatatatatttatttaaaatggttttgaatttttcctattctggttgatattgcccaaaagctattcagt'
VMA16_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_dw['Primer_up_U']='U2' #user site up
VMA16_dw['Primer_down_U']='GV2'#user site down
VMA16_dw['Primer_up']=''
VMA16_dw['Primer_up_Tm']=''
VMA16_dw['Primer_down']=''
VMA16_dw['Primer_down_Tm']=''





CAX4_up={} 
CAX4_up['name']='CAX4_up'
CAX4_up['seq']='aaccccgagaatttaaatagaacagagtatatgcatttattaacagttattacgccttgtccccagggagagtagtaagggatatgggcatttataaatgtacataagtcacgtttttgcagccaataaaatataataatagaatagtaacaaggaaagctaggccaaaactccttacatggagctagtatacccctttt'
CAX4_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_up['Primer_up_U']='GV1' #user site up
CAX4_up['Primer_down_U']='U1'#user site down
CAX4_up['Primer_up']=''
CAX4_up['Primer_up_Tm']=''
CAX4_up['Primer_down']=''
CAX4_up['Primer_down_Tm']=''

CAX4_dw={}
CAX4_dw['name']='CAX4_dw'
CAX4_dw['seq']='aagtgtgggtgacatgctaaatccgcggctttccttaagctttttcacaaaccatctattatgtccagaagctccgctgattatccgtggaaaagggtcttattctgccgtaggaaacttcttctcgagaaaaaagtgagttcacccgcggcgctctcgtaaaacccgagggccgattctttccctccgtggaacaatcg'
CAX4_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_dw['Primer_up_U']='U2' #user site up
CAX4_dw['Primer_down_U']='GV2'#user site down
CAX4_dw['Primer_up']=''
CAX4_dw['Primer_up_Tm']=''
CAX4_dw['Primer_down']=''
CAX4_dw['Primer_down_Tm']=''



PRO2_up={} #PPA1
PRO2_up['name']='PRO2_up'
PRO2_up['seq']='ttgtaaagtgaccttgttttgactttttaacgagtaaaaaaacgagttaacaaattcttgacgtaaacgataaagccgcccatcaaaagaaatgcaataatgtcaatgatgaaatacctatacaggtagatggatttcgtgttgttcttcgatttatttaacacgtttttcagctctatcaaactatccatttcccttag'
PRO2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_up['Primer_up_U']='GV1' #user site up
PRO2_up['Primer_down_U']='U1'#user site down
PRO2_up['Primer_up']=''
PRO2_up['Primer_up_Tm']=''
PRO2_up['Primer_down']=''
PRO2_up['Primer_down_Tm']=''

PRO2_dw={}
PRO2_dw['name']='PRO2_dw'
PRO2_dw['seq']='gagtagatgtaagacgtgaaaatgccatggtaaaacgtattgtttagtgaaatgtatactaagtaggagtaaaaatcagcaatttctggtgttgaaatgtgaacaacagcgttctcctccgaaaaagcaagctataacgttgctcaacttttgattccgaaatcagaagcttaaataaggaaaactgaaagctaattgga'
PRO2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_dw['Primer_up_U']='U2' #user site up
PRO2_dw['Primer_down_U']='GV2'#user site down
PRO2_dw['Primer_up']=''
PRO2_dw['Primer_up_Tm']=''
PRO2_dw['Primer_down']=''
PRO2_dw['Primer_down_Tm']=''



GLR1_up={} #PPA1
GLR1_up['name']='GLR1_up'
GLR1_up['seq']='tggtcaattctatgccttttatgcacattgcatgagtattctaaaaagatgctgcatgctgcccgtaaatcttcattattttcagagtcaggtacggaaaagacgacagtttctccgtataacagcattgaaagcgtggaagaatcaaactcggctctagattttacgcgtttagcataaaattgttggttttatacgta'
GLR1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_up['Primer_up_U']='GV1' #user site up
GLR1_up['Primer_down_U']='U1'#user site down
GLR1_up['Primer_up']=''
GLR1_up['Primer_up_Tm']=''
GLR1_up['Primer_down']=''
GLR1_up['Primer_down_Tm']=''

GLR1_dw={}
GLR1_dw['name']='GLR1_dw'
GLR1_dw['seq']='cttttactgctttctaactttaagccttcaaagaagaagctcttctctttctgatttcagccttctcagcctttctttcagacaatctcttagccaacaattgagcgtattcggcagcagcttctctttgagcttgagcgtttctgaccttcaaagctctttggtgtctctttctttgcaatctttgaggagtaaccaatc'
GLR1_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_dw['Primer_up_U']='U2' #user site up
GLR1_dw['Primer_down_U']='GV2'#user site down
GLR1_dw['Primer_up']=''
GLR1_dw['Primer_up_Tm']=''
GLR1_dw['Primer_down']=''
GLR1_dw['Primer_down_Tm']=''



YSY6_up={} 
YSY6_up['name']='YSY6_up'
YSY6_up['seq']='ggagtcttcttgcgagaaaagaaataattataatataaaaaaaaattgaatggagataagaagtttaactagcaaagttcgagagatataaaaagtagagatgcgaaaataaaattaaaacaaatctgaaaataatggaagtgaaacaaggctaagcaagaaaattttctatattgccaagacaggaacgtttgtgatga'
YSY6_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_up['Primer_up_U']='GV1' #user site up
YSY6_up['Primer_down_U']='U1'#user site down
YSY6_up['Primer_up']=''
YSY6_up['Primer_up_Tm']=''
YSY6_up['Primer_down']=''
YSY6_up['Primer_down_Tm']=''

YSY6_dw={}
YSY6_dw['name']='YSY6_dw'
YSY6_dw['seq']='gcagtattacccgcgttgtcatttatggatgcgaaaaaagatgaagtaattcaagtgctattgtgatttaccatgccaatcaaacgtgcaacaaagatgagtatgaggggggacccaggcatagtacacagaaaagctttgtagactgtctttgttttcttaatttttcaagtaggctttctgcttttcttctgtactta'
YSY6_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_dw['Primer_up_U']='U2' #user site up
YSY6_dw['Primer_down_U']='GV2'#user site down
YSY6_dw['Primer_up']=''
YSY6_dw['Primer_up_Tm']=''
YSY6_dw['Primer_down']=''
YSY6_dw['Primer_down_Tm']=''





PDX3_up={} 
PDX3_up['name']='PDX3_up'
PDX3_up['seq']='aggtagagtatctttaaaatatctaatcatgcaaattagtcatttcttctttcttataagttgcggccccttctagtatttacaatttacccggcgctggttctggtggcatctataagagctctatctggctgctaagcagacttgctgttacttgtatgtgctgtttatcaagattggggcattacgcatgtgtatgt'
PDX3_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PDX3_up['Primer_up_U']='GV1' #user site up
PDX3_up['Primer_down_U']='U1'#user site down
PDX3_up['Primer_up']=''
PDX3_up['Primer_up_Tm']=''
PDX3_up['Primer_down']=''
PDX3_up['Primer_down_Tm']=''

PDX3_dw={}
PDX3_dw['name']='PDX3_dw'
PDX3_dw['seq']='tctttggatgaatttgtcggttctaactcttttttggaaacggcaaggtgagcagtaggcagctcatctcttcaactatttttataacatataatttccctttgtgaaaaaattttcccgccatttccaaatttcggaattcctttttcttttttttcatctcggtatattctgacatattaccctataggcggtataaa'
PDX3_dw['need_reverse']= 1 #if parts needs to be reversed:1 else:0
PDX3_dw['Primer_up_U']='U2' #user site up
PDX3_dw['Primer_down_U']='GV2'#user site down
PDX3_dw['Primer_up']=''
PDX3_dw['Primer_up_Tm']=''
PDX3_dw['Primer_down']=''
PDX3_dw['Primer_down_Tm']=''



Parts=[KanMX,KGD2_up,KGD2_dw,HEM25_up,HEM25_dw,VMA16_up,VMA16_dw,
       CAX4_up,CAX4_dw,PRO2_up,PRO2_dw,GLR1_up,GLR1_dw,PDX3_up,PDX3_dw,YSY6_up,YSY6_dw]


In [205]:
# Fit and Yellow candidate
#PRY1
#GET4
#YLR271W 
#YCR101C
#STV1

PRY1_up={} 
PRY1_up['name']='PRY1_up'
PRY1_up['seq']='ttctagtatctacactgaagtcaacgccgcaactatcattaaccccggagaaacatcttctctcgctagcgatttcgccacatctgaaaagccaaacgagcccacttctgtcaaatccacctcaaacgaaggcacctcttccacaacaacaacctaccaacagactgttgctacactgtatgccaagccctccagcacaa'
PRY1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
PRY1_up['Primer_up_U']='GV1' #user site up
PRY1_up['Primer_down_U']='U1'#user site down
PRY1_up['Primer_up']=''
PRY1_up['Primer_up_Tm']=''
PRY1_up['Primer_down']=''
PRY1_up['Primer_down_Tm']=''

PRY1_dw={} 
PRY1_dw['name']='PRY1_dw'
PRY1_dw['seq']='gaaaaattttcgctcattttcgcgaattgttgttggcgacactcgacaatttcaatttaatgtagctcttaaagacttgcaacggatagtatatttactaatttatagtttttacatttcgtacagaacttccttcgaaacattggctattagcttttcaattgaacgtctttgaaaaagaacgtctaagtacacaacagca'
PRY1_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
PRY1_dw['Primer_up_U']='U2' #user site up
PRY1_dw['Primer_down_U']='GV2'#user site down
PRY1_dw['Primer_up']=''
PRY1_dw['Primer_up_Tm']=''
PRY1_dw['Primer_down']=''
PRY1_dw['Primer_down_Tm']=''

GET4_up={} 
GET4_up['name']='GET4_up'
GET4_up['seq']='aaagtatgtcaaccgtcgagtgagttagtagcccgtttgttgaaataagtcaataattatttaccatcaaaatacacctttcagtattttgttcgttttgtcgctcttcgttaaggtaacaaacaatgggaaaaaaagtgaaaggaaagtcaaaaaaatgaaataataacatgtgcggtaaaacaaccaagtaaacatca'
GET4_up['need_reverse']= 0#if parts needs to be reversed:1 else:0
GET4_up['Primer_up_U']='GV1' #user site up
GET4_up['Primer_down_U']='U1'#user site down
GET4_up['Primer_up']=''
GET4_up['Primer_up_Tm']=''
GET4_up['Primer_down']=''
GET4_up['Primer_down_Tm']=''

GET4_dw={} 
GET4_dw['name']='GET4_dw'
GET4_dw['seq']='aaagagcatatttgcgaaggaatagataaatatgtttgcggtggcttgacgaccatgacgtatatacatatatatgtatgtgctagctatttgtcgtctttaatgatagcagacctattaagggcctccaacaattcaggcctcttggcgtctatcaagttttgtttagcttctgtataagagtatagctttctatgtct' #
GET4_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
GET4_dw['Primer_up_U']='U2' #user site up
GET4_dw['Primer_down_U']='GV2'#user site down
GET4_dw['Primer_up']=''
GET4_dw['Primer_up_Tm']=''
GET4_dw['Primer_down']=''
GET4_dw['Primer_down_Tm']=''





CMG1_up={} 
CMG1_up['name']='CMG1_up'
CMG1_up['seq']='ggttgttaaaattctcacatcatccacctttttcctactctcattttctttttttgaaagttgtttgttttatataccttagtttcacaacaacataaggcatgcctatttgagattttgtgattaaaaaataatagaatacgtactatataaccgtcggtttaaagtaaataaattaaagactaagaatgaaacaatta'
CMG1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
CMG1_up['Primer_up_U']='GV1' #user site up
CMG1_up['Primer_down_U']='U1'#user site down
CMG1_up['Primer_up']=''
CMG1_up['Primer_up_Tm']=''
CMG1_up['Primer_down']=''
CMG1_up['Primer_down_Tm']=''


CMG1_dw={} 
CMG1_dw['name']='CMG1_dw'
CMG1_dw['seq']='gatgattttcttgaaactctccttacccaataaatcatctgacgataaattactaaagatatcgatgaatccattataaatggcgttgtccttggatgctaactcggtgaaaaataatctgcacatatcacttattccctggtctggattgtctaggcatttggccatttctcctaattgtcctttaaccttaacctgtc'
CMG1_dw['need_reverse']=0
CMG1_dw['Primer_up_U']='U2' #user site up
CMG1_dw['Primer_down_U']='GV2'#user site down
CMG1_dw['Primer_up']=''
CMG1_dw['Primer_up_Tm']=''
CMG1_dw['Primer_down']=''
CMG1_dw['Primer_down_Tm']=''

YCR101C_up={}
YCR101C_up['name']='YCR101C_up'
YCR101C_up['seq']='tacctataatagcggctcgctgtccgaacgattgcgccgcacttttctgcctcttacattcccttttcctaaggcgataagcaggctgaatggaaactacgatataacgcagtagaggtcccccaagagctggaaagaaaatattggtttgatagccagcactattctatccagtgggctcttgtaccattttctgtggt'
YCR101C_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YCR101C_up['Primer_up_U']='GV1' #user site up
YCR101C_up['Primer_down_U']='U1'#user site down
YCR101C_up['Primer_up']=''
YCR101C_up['Primer_up_Tm']=''
YCR101C_up['Primer_down']=''
YCR101C_up['Primer_down_Tm']=''


YCR101C_dw={} 
YCR101C_dw['name']='YCR101C_dw'
YCR101C_dw['seq']='aatgatacttcgatactctgtttgttctctaaccgtgaatacgatgaagaatatcgcttaactgctacctacacacagttggctttcagtagtgatggaggcaaaacgtttaaagattttgatgaatttaaagataagattgttactcaatacaagatattgaagtcatatgtggtcgttttaacgcaagatgacgggca'
YCR101C_dw['need_reverse']=0
YCR101C_dw['Primer_up_U']='U2' #user site up
YCR101C_dw['Primer_down_U']='GV2'#user site down
YCR101C_dw['Primer_up']=''
YCR101C_dw['Primer_up_Tm']=''
YCR101C_dw['Primer_down']=''
YCR101C_dw['Primer_down_Tm']=''

STV1_up={}
STV1_up['name']='STV1_up'
STV1_up['seq']='tattaataggtgcttcttccagggtctagttcattacctcccctaaatatcattctcagatcattatagcttctatttcctgttatgcgggtagcggggcttatttttgtttctcaaaatagaggtgtccttttttgccatttttgtttatgtagtcaactgcataggtagttaagaaaagacataggcccacgaaggtg'
STV1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
STV1_up['Primer_up_U']='GV1' #user site up
STV1_up['Primer_down_U']='U1'#user site down
STV1_up['Primer_up']=''
STV1_up['Primer_up_Tm']=''
STV1_up['Primer_down']=''
STV1_up['Primer_down_Tm']=''


STV1_dw={} 
STV1_dw['name']='STV1_dw'
STV1_dw['seq']='cacatagtttacggtatatatatgtctgggtcggtcaagtggtctaccaataggtcataaatttgagcagggatcttagcgatgaacccgacgcccagccgaatgatctcgtcagcatcaaaatccggaaactgtcgtagtaaattaacaggggaatccgatttgaaaaccttggatctcattttaactaggaatgccac'
STV1_dw['need_reverse']=0
STV1_dw['Primer_up_U']='U2' #user site up
STV1_dw['Primer_down_U']='GV2'#user site down
STV1_dw['Primer_up']=''
STV1_dw['Primer_up_Tm']=''
STV1_dw['Primer_down']=''
STV1_dw['Primer_down_Tm']=''


Parts=[PRY1_up,PRY1_dw,GET4_up,GET4_dw,CMG1_up,CMG1_dw,YCR101C_up,YCR101C_dw,STV1_up,STV1_dw]


In [206]:
#fills the part dicts with primers:

dt.User_Assebmly(Parts, 50, User_Overhangs)




Will use input Tm_target for design:  50


[{'name': 'PRY1_up',
  'seq': 'TTCTAGTATCTACACTGAAGTCAACGCCGCAACTATCATTAACCCCGGAGAAACATCTTCTCTCGCTAGCGATTTCGCCACATCTGAAAAGCCAAACGAGCCCACTTCTGTCAAATCCACCTCAAACGAAGGCACCTCTTCCACAACAACAACCTACCAACAGACTGTTGCTACACTGTATGCCAAGCCCTCCAGCACAA',
  'need_reverse': 0,
  'Primer_up_U': 'GV1',
  'Primer_down_U': 'U1',
  'Primer_up': 'CGTGCGAuTTCTAGTATCTACACTGAAGTCAA',
  'Primer_up_Tm': 50.06102982291435,
  'Primer_down': 'AAGAGGGCuTTGTGCTGGAGGGCT',
  'Primer_down_Tm': 50.564760859970704},
 {'name': 'PRY1_dw',
  'seq': 'GAAAAATTTTCGCTCATTTTCGCGAATTGTTGTTGGCGACACTCGACAATTTCAATTTAATGTAGCTCTTAAAGACTTGCAACGGATAGTATATTTACTAATTTATAGTTTTTACATTTCGTACAGAACTTCCTTCGAAACATTGGCTATTAGCTTTTCAATTGAACGTCTTTGAAAAAGAACGTCTAAGTACACAACAGCA',
  'need_reverse': 0,
  'Primer_up_U': 'U2',
  'Primer_down_U': 'GV2',
  'Primer_up': 'ACACAGGCuGAAAAATTTTCGCTCATTTTCGC',
  'Primer_up_Tm': 50.997092976344845,
  'Primer_down': 'CACGCGAuTGCTGTTGTGTACTTAGACG',
  'Primer_down_Tm': 50.922849712901154},
 {'name': 'GET4_up',
  'seq': 'AAAGTA

{'pdx3': [{'name': 'PDX3_up',
   'seq': 'AGGTAGAGTATCTTTAAAATATCTAATCATGCAAATTAGTCATTTCTTCTTTCTTATAAGTTGCGGCCCCTTCTAGTATTTACAATTTACCCGGCGCTGGTTCTGGTGGCATCTATAAGAGCTCTATCTGGCTGCTAAGCAGACTTGCTGTTACTTGTATGTGCTGTTTATCAAGATTGGGGCATTACGCATGTGTATGT',
   'need_reverse': 1,
   'Primer_up_U': 'GV1',
   'Primer_down_U': 'U1',
   'Primer_up': 'AAGAGGGCuAGGTAGAGTATCTTTAAAATATCTAATCA',
   'Primer_up_Tm': 50.99909277252226,
   'Primer_down': 'CGTGCGAuACATACACATGCGTAATGCC',
   'Primer_down_Tm': 52.637722030437715},
  {'name': 'pTEF::KanMX::tTef',
   'seq': 'TAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTAAGGAAAAGACTCACGTTTCGAGGCCGCGATTAAATTCCAACATGGAT

In [207]:



primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]
i=108

for part in Parts:
    #here we adapt the order with which the overhang names are appended to match the Overhang names to the inversion of sequences
    if part['need_reverse']==0:
        primer_OH.append(part['Primer_up' +'_U'])
        primer_OH.append(part['Primer_down'+'_U'])
    elif part['need_reverse']==1:
        primer_OH.append(part['Primer_down'+'_U'])
        primer_OH.append(part['Primer_up' +'_U'])

    for primer in ['Primer_up', 'Primer_down']:
        primer_name.append('BTX.1')#.zfill fills primer number wiht leading 0
        primer_seq.append(part[primer])
        primer_tm.append(round(part[primer +'_Tm'],2))
        primer_template_len.append(len(part['seq']))
        primer_template_name.append(part['name'])
        #primer_OH.append(part[primer +'_U'])
        primer_dir.append('fwd'  if primer[7:] == 'up' else 'rvs' )
        benchling.append(part[primer].replace('u','t'))
        i+=1
## need to fix the current overhang name bug (if seq needs inversion the OH name doenst match)
        


df = np.stack([primer_name, primer_seq,primer_dir, primer_tm,primer_OH, primer_template_name, primer_template_len, benchling] ).T
Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Table.set_index('Primer Name')

,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
Primer Name,,,,,,,
BTX.1,CGTGCGAuTTCTAGTATCTACACTGAAGTCAA,fwd,50.06,GV1,PRY1_up,200,CGTGCGAtTTCTAGTATCTACACTGAAGTCAA
BTX.1,AAGAGGGCuTTGTGCTGGAGGGCT,rvs,50.56,U1,PRY1_up,200,AAGAGGGCtTTGTGCTGGAGGGCT
BTX.1,ACACAGGCuGAAAAATTTTCGCTCATTTTCGC,fwd,51.0,U2,PRY1_dw,202,ACACAGGCtGAAAAATTTTCGCTCATTTTCGC
BTX.1,CACGCGAuTGCTGTTGTGTACTTAGACG,rvs,50.92,GV2,PRY1_dw,202,CACGCGAtTGCTGTTGTGTACTTAGACG
BTX.1,CGTGCGAuAAAGTATGTCAACCGTCGAG,fwd,50.97,GV1,GET4_up,200,CGTGCGAtAAAGTATGTCAACCGTCGAG
BTX.1,AAGAGGGCuTGATGTTTACTTGGTTGTTTTACC,rvs,52.12,U1,GET4_up,200,AAGAGGGCtTGATGTTTACTTGGTTGTTTTACC
BTX.1,ACACAGGCuAAAGAGCATATTTGCGAAGGA,fwd,51.16,U2,GET4_dw,200,ACACAGGCtAAAGAGCATATTTGCGAAGGA
BTX.1,CACGCGAuAGACATAGAAAGCTATACTCTTATAC,rvs,50.23,GV2,GET4_dw,200,CACGCGAtAGACATAGAAAGCTATACTCTTATAC
BTX.1,CGTGCGAuGGTTGTTAAAATTCTCACATCATCC,fwd,52.46,GV1,CMG1_up,200,CGTGCGAtGGTTGTTAAAATTCTCACATCATCC


BTX.161
BTX.162
BTX.163
BTX.164
BTX.165
BTX.166
BTX.167
BTX.168
BTX.169
BTX.170
BTX.171
BTX.172
BTX.173
BTX.174
BTX.175
BTX.176
BTX.177
BTX.178
BTX.179


# Design upstream and downstream confirmation primers 

In [220]:

genes = ['cax4','glr1','hem25','kgd2','pdx3','pro2','vma16','ysy6']

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/Betaxanthin/DNA_parts/Overexpression/'

#store file name
part_file={}
for gene in genes:
    part_file[gene]= gene+".gb"
part_seq={}
for gene in genes:
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    part_seq[gene] = str(recs[0].seq)

In [221]:
genes = ['pry1','get4','cmg1','ycr101c','stv1']

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/Betaxanthin/DNA_parts/KO_confirmations/second_batch/'
os.listdir(path2parts)
part_file={}
for gene in genes:
    part_file[gene]= gene+".gb"
    
part_seq={}
for gene in genes:
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    part_seq[gene] = str(recs[0].seq)

In [222]:

primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]

for gene in genes:

    #find user primer sequence in table:
    i_primer_up = (Table['Template'].str.contains(gene.upper()) ) & ( Table['User'].str.contains('GV1') )
    i_primer_dw = (Table['Template'].str.contains(gene.upper()) ) & ( Table['User'].str.contains('GV2') )
    
    primer_up =  Table[i_primer_up]['Seq'].values[0]
    primer_dw =  Table[i_primer_dw]['Seq'].values[0]

    #remove user overhang
    primer_up=  primer_up[primer_up.find('u')+1:]
    primer_dw=  primer_dw[primer_dw.find('u')+1:]
    #find the upper position of the user primers
    #if the gene sequence used was in the reverse direction the up primer wont be found 
    #in that case the position of the down primers will give the upper position
    
    pos_up = part_seq[gene].find(primer_up)
    if pos_up == -1:
        #if primer up is not found it is because sequence of the gene was in the reverse orientation
        #primer_up will therefore be downstream of the sequence
        pos_dw = part_seq[gene].find(dt.reverse_complement(primer_up))#to find reverse primer we need to search its unreversed complement
        pos_up = part_seq[gene].find(primer_dw)
    else :
        pos_dw = part_seq[gene].find(dt.reverse_complement(primer_dw))#to find reverse primer we need to search its unreversed complement
        

    
    valid_up = dt.make_anneal(part_seq[gene][pos_up-150:pos_up], 'F', 50 )
    
    valid_dw = dt.make_anneal(part_seq[gene][pos_dw+150:pos_dw+250], 'R', 50 )
    primer_name.append('BTX.')
    primer_name.append('BTX.')
    primer_dir.append('fw')
    primer_dir.append('rv')
    
    primer_tm.append(round(valid_up[1],2))
    primer_tm.append(round(valid_dw[1],2))
    primer_seq.append(valid_up[0])
    primer_seq.append(valid_dw[0])
    primer_OH.append('up.out')
    primer_OH.append('dw.out')
    
    primer_template_len.append(pos_dw+250 -(pos_up-150))
    primer_template_len.append(pos_dw+250 -(pos_up-150))
    primer_template_name.append(gene)
    primer_template_name.append(gene)

    
    
df = np.stack([primer_name, primer_seq,primer_dir, primer_tm, primer_OH,primer_template_name, primer_template_len] ).T
Confirmation_Primers=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','position', 'Template', 'Template_Len'])

Confirmation_Primers.set_index('Primer Name')

,Seq,Dir,Tm (c),position,Template,Template_Len
Primer Name,,,,,,
BTX.,AACAATCTCGACATCTTGTAGC,fw,51.24,up.out,pry1,2778
BTX.,GGCGAATCAATGGTAGTTAC,rv,50.86,dw.out,pry1,2778
BTX.,CCACTCGATTGAGAACCG,fw,50.88,up.out,get4,1742
BTX.,CCCAACGAGAAACTTGGG,rv,52.51,dw.out,get4,1742
BTX.,GACCAGAAGGCTACATGAA,fw,50.25,up.out,cmg1,2209
BTX.,TTATCATCAAAATATTGTGAAAAGAGC,rv,50.46,dw.out,cmg1,2209
BTX.,TATATCCTTACTATCTACTATCCAAC,fw,50.7,up.out,ycr101c,2185
BTX.,CATTCAACAGGTGAGAATTTCAA,rv,50.22,dw.out,ycr101c,2185
BTX.,TTTGCATCTTTTTGTCCACG,fw,50.16,up.out,stv1,3765


In [223]:
for i in range(180,200):
    print('BTX.' +str(i))

BTX.180
BTX.181
BTX.182
BTX.183
BTX.184
BTX.185
BTX.186
BTX.187
BTX.188
BTX.189
BTX.190
BTX.191
BTX.192
BTX.193
BTX.194
BTX.195
BTX.196
BTX.197
BTX.198
BTX.199


In [200]:
for i in range(148,170):
    print('BTX.' +str(i))

BTX.148
BTX.149
BTX.150
BTX.151
BTX.152
BTX.153
BTX.154
BTX.155
BTX.156
BTX.157
BTX.158
BTX.159
BTX.160
BTX.161
BTX.162
BTX.163
BTX.164
BTX.165
BTX.166
BTX.167
BTX.168
BTX.169


In [36]:
#find
for gene in genes:

{'cax4': 'GGGTGATTATGAAGATGAGTAAGATGCGAATGGATATGAGGATATGAATCTTTTTAGAGACAATGAAGATGCAAAGAAGTAGTTAAGCACATTGACCTCAGGGTTGCTATATGCTACTTTTAGTCCGTTGGATTTCTGAAGCAGTATTGGCCACAAAAATTTCTTCATTATTTCTACGATTAGAGTTTCTTCTGGATTTTTCTCGTCCCTGAGACTTACTCCTTTTCAAGTCATTTTCAGCAGCAGCTTTGATTAAATCATATCTTGTGATTTTACTTGGAGAGCTATTGTGATGGGAACTTTGTCTGCATGTGACTGTATTAGTCAGAGATGTTATGGGCTTAACCATAAGGGAAGAAACTATGGCACGCATGAATGAATTACTGCTTTTACTGCTGTCATCTGTTGAGTTTGCTGAGTCTTCCGTTCTTGTAGTCTTGGCTGGTGTGAGTAACATTACGTATTGTTTCTGTAGAGAAGATAACTTATAACTTCCGTCTTGTCACTAGTTGTTTTAATTATATGTTTGTTACAAGTTTTTATGTATAGATATATGTTACAACTTTTAGAAGTATTGTGGACATAACAGAGAGAAGGGAATGTTGCTATATTTATATATTTTATATATTGCCCTCAAAACTGTATCTCATCTTTTTTTCTGCAATCCTTGCTTTCGTAATCTAAGTGTGGGTGACATGCTAAATCCGCGGCTTTCCTTAAGCTTTTTCACAAACCATCTATTATGTCCAGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTAGGAAACTTCTTCTCGAGAAAAAAGTGAGTTCACCCGCGGCGCTCTCGTAAAACCCGAGGGCCGATTCTTTCCCTCCGTGGAACAATCGGCCCCGCGGCGTCAGAGGGTATTATCTCCGCAAGTGATAGAACTCACATTTATTCTACGTTCTGCATGGTTTTGATTATCTTCTTTTGCACGATGTCACATACTTTTT

In [ ]:
start=0
    stop=0
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    for rec in recs:
        #loop through feature to features to find the location  of the CDS:
        for feat in rec.features:
            if gene + ' gene'in feat.qualifiers['label'] or gene in feat.qualifiers['label']:
                location= feat.location
                start = location.start.position
                stop = location.end.position

        #stores seq in dict
        part_seq[gene] = str(rec.seq[start-homology_length-out_gap:stop+homology_length+out_gap])

In [49]:
CAX4_up['seq'].upper() in part_seq['cax4']

True

'GGGTGATTATGAAGATGAGTAAGATGCGAATGGATATGAGGATATGAATCTTTTTAGAGACAATGAAGATGCAAAGAAGTAGTTAAGCACATTGACCTCAGGGTTGCTATATGCTACTTTTAGTCCGTTGGATTTCTGAAGCAGTATTGGCCACAAAAATTTCTTCATTATTTCTACGATTAGAGTTTCTTCTGGATTTTTCTCGTCCCTGAGACTTACTCCTTTTCAAGTCATTTTCAGCAGCAGCTTTGATTAAATCATATCTTGTGATTTTACTTGGAGAGCTATTGTGATGGGAACTTTGTCTGCATGTGACTGTATTAGTCAGAGATGTTATGGGCTTAACCATAAGGGAAGAAACTATGGCACGCATGAATGAATTACTGCTTTTACTGCTGTCATCTGTTGAGTTTGCTGAGTCTTCCGTTCTTGTAGTCTTGGCTGGTGTGAGTAACATTACGTATTGTTTCTGTAGAGAAGATAACTTATAACTTCCGTCTTGTCACTAGTTGTTTTAATTATATGTTTGTTACAAGTTTTTATGTATAGATATATGTTACAACTTTTAGAAGTATTGTGGACATAACAGAGAGAAGGGAATGTTGCTATATTTATATATTTTATATATTGCCCTCAAAACTGTATCTCATCTTTTTTTCTGCAATCCTTGCTTTCGTAATCTAAGTGTGGGTGACATGCTAAATCCGCGGCTTTCCTTAAGCTTTTTCACAAACCATCTATTATGTCCAGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTAGGAAACTTCTTCTCGAGAAAAAAGTGAGTTCACCCGCGGCGCTCTCGTAAAACCCGAGGGCCGATTCTTTCCCTCCGTGGAACAATCGGCCCCGCGGCGTCAGAGGGTATTATCTCCGCAAGTGATAGAACTCACATTTATTCTACGTTCTGCATGGTTTTGATTATCTTCTTTTGCACGATGTCACATACTTTTTCGATTACCG